# __Домашняя работа 2__

In [ ]:
pip install implicit

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
data = pd.read_csv('./retail_train.csv')
data.head(10)

In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(5)

In [ ]:
def weighted_random_recommendation(items, items_weights, n=5):
    """Случайные рекомендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items)
    items_weights = np.array(items_weights)
    recs = np.random.choice(items, n, replace=False, p = items_weights)
    
    return recs.tolist()

In [ ]:
data_train['probability'] = 1 / len(data_train)

In [ ]:
items = data_train.item_id.unique()
items_weights = data_train.groupby(['item_id']).sum(['probability'])

In [ ]:
items_weights = items_weights['probability']

In [ ]:
items_weights

In [ ]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, items_weights, n=5))
result.head(10)

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [ ]:
result = pd.read_csv('./predictions_basic.csv')
result.head(5)

In [ ]:
def precision_at_k(result_recomendation, actual_recomendation, k=5):
    # get top-k recommendations from result_recomendation
    top_k = result_recomendation[:k]
    print(top_k)
    # calculate the number of correct recommendations
    num_correct = len(set(top_k).intersection(actual_recomendation))
    print(num_correct)
    print(set(top_k))
    print(set(actual_recomendation))
    # calculate precision@k
    precision = num_correct / k
    return precision

In [ ]:
precision_at_k(result.iloc[0]['random_recommendation'], result.iloc[0]['actual'])

In [ ]:
result.iloc[0]['actual']

In [ ]:
# Очистим 'actual' посмотрим, что получаетя.
result['actual'] = result['actual'].apply(lambda x: [int(el) for el in x.replace('[', '').replace(']', '').split()])

In [ ]:
def precision_at_k(result_recomendation, actual_recomendation, k=5):
    # get top-k recommendations from result_recomendation
    top_k = result_recomendation[:k]
    # calculate the number of correct recommendations
    num_correct = len(set(top_k).intersection(actual_recomendation))
    # calculate precision@k
    precision = num_correct / k
    return precision

In [ ]:
result.apply(lambda row: precision_at_k(eval(row['random_recommendation']), row['actual']), axis=1).mean()

In [ ]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

In [ ]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

In [ ]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()

In [ ]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()

In [ ]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code